In [141]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics

%matplotlib inline

In [142]:
data = pd.read_csv("Dados/LocTreino_Equipe_4.csv")

bts_data = pd.read_csv("Dados/dados_BTSs.csv")

In [275]:
data.head()

,pontoId,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
0,397,-8.075475,-34.896290,112.990000,114.390000,128.640000,114.640000,128.09,133.740000,1,0.0,2,0.0,2,2
1,792,-8.072956,-34.896622,92.123333,98.890000,120.490000,91.990000,122.59,120.423333,1,0.0,2,0.0,2,1
2,864,-8.072692,-34.896423,102.323333,109.190000,122.556667,99.790000,122.99,119.323333,1,0.0,2,0.0,1,1
3,717,-8.073347,-34.900574,123.890000,121.823333,124.656667,129.490000,141.89,140.990000,1,1.0,1,1.0,2,2
4,2826,-8.062634,-34.888382,120.856667,124.956667,142.290000,126.156667,132.39,138.090000,1,2.0,4,2.0,0,1


In [323]:
data.drop_duplicates(inplace=True)
data.shape
data.drop(['pontoId'], axis=1)
data.describe()

,pontoId,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0000
mean,1471.292667,-8.070097,-34.894814,116.654370,119.101165,123.863545,115.814052,121.320526,125.656192,0.663333,1.008733,2.525333,1.099388,1.093333,1.0560
std,849.057388,0.004232,0.004354,15.229664,12.666188,9.031316,12.414620,12.552702,13.134365,0.698317,0.863886,0.967117,1.954747,0.856998,0.6692
min,2.000000,-8.077570,-34.906830,86.256667,77.256667,88.340000,84.056667,79.523333,89.856667,0.000000,0.000000,1.000000,0.000000,0.000000,0.0000
25%,746.750000,-8.073197,-34.897387,106.827500,111.673333,119.190000,106.583750,112.823333,116.077500,0.000000,0.000000,2.000000,0.000000,0.000000,1.0000
50%,1464.500000,-8.069940,-34.895346,117.156667,120.690000,124.273333,116.273333,122.840000,128.756667,1.000000,1.000000,3.000000,1.000000,1.000000,1.0000
75%,2190.250000,-8.067113,-34.891169,124.790000,128.808750,130.117083,125.590000,130.423333,135.996250,1.000000,1.000000,3.000000,1.000000,2.000000,2.0000
max,2955.000000,-8.059339,-34.885067,476.882449,144.890000,145.890000,145.423333,146.890000,150.123333,3.000000,11.090452,5.000000,38.747484,3.000000,2.0000


In [145]:
data.isnull().sum()

pontoId     0
lat         0
lon         0
pathBTS1    0
pathBTS2    0
pathBTS3    0
pathBTS4    0
pathBTS5    0
pathBTS6    0
taBTS1      0
taBTS2      1
taBTS3      0
taBTS4      1
taBTS5      0
taBTS6      0
dtype: int64

In [146]:
data['taBTS2'] = data['taBTS2'].fillna(data['taBTS2'].mean())
data['taBTS4'] = data['taBTS4'].fillna(data['taBTS4'].mean())

In [147]:
bts_data.describe()

,lat,lon,bcch,eirp
count,6.000000,6.000000,6.000000,6.000000e+00
mean,-8.071190,-34.895662,696.500000,5.559000e+01
std,0.005477,0.006370,54.617763,7.783606e-15
min,-8.076361,-34.908000,662.000000,5.559000e+01
25%,-8.075917,-34.894611,667.750000,5.559000e+01
50%,-8.072139,-34.894597,677.500000,5.559000e+01
75%,-8.066590,-34.893187,688.000000,5.559000e+01
max,-8.064583,-34.889444,806.000000,5.559000e+01


In [8]:
data.describe()

,pontoId,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1499.000000,1500.000000,1499.000000,1500.000000,1500.0000
mean,1471.292667,-8.070097,-34.894814,116.654370,119.101165,123.863545,115.814052,121.320526,125.656192,0.663333,1.008733,2.525333,1.099388,1.093333,1.0560
std,849.057388,0.004232,0.004354,15.229664,12.666188,9.031316,12.414620,12.552702,13.134365,0.698317,0.864175,0.967117,1.955399,0.856998,0.6692
min,2.000000,-8.077570,-34.906830,86.256667,77.256667,88.340000,84.056667,79.523333,89.856667,0.000000,0.000000,1.000000,0.000000,0.000000,0.0000
25%,746.750000,-8.073197,-34.897387,106.827500,111.673333,119.190000,106.583750,112.823333,116.077500,0.000000,0.000000,2.000000,0.000000,0.000000,1.0000
50%,1464.500000,-8.069940,-34.895346,117.156667,120.690000,124.273333,116.273333,122.840000,128.756667,1.000000,1.000000,3.000000,1.000000,1.000000,1.0000
75%,2190.250000,-8.067113,-34.891169,124.790000,128.808750,130.117083,125.590000,130.423333,135.996250,1.000000,1.000000,3.000000,1.000000,2.000000,2.0000
max,2955.000000,-8.059339,-34.885067,476.882449,144.890000,145.890000,145.423333,146.890000,150.123333,3.000000,11.090452,5.000000,38.747484,3.000000,2.0000


In [276]:
import xgboost as xgb

In [295]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [296]:
x = data.drop(['lat', 'lon', 'pontoId'], axis=1)
x.describe()

,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0000
mean,116.654370,119.101165,123.863545,115.814052,121.320526,125.656192,0.663333,1.008733,2.525333,1.099388,1.093333,1.0560
std,15.229664,12.666188,9.031316,12.414620,12.552702,13.134365,0.698317,0.863886,0.967117,1.954747,0.856998,0.6692
min,86.256667,77.256667,88.340000,84.056667,79.523333,89.856667,0.000000,0.000000,1.000000,0.000000,0.000000,0.0000
25%,106.827500,111.673333,119.190000,106.583750,112.823333,116.077500,0.000000,0.000000,2.000000,0.000000,0.000000,1.0000
50%,117.156667,120.690000,124.273333,116.273333,122.840000,128.756667,1.000000,1.000000,3.000000,1.000000,1.000000,1.0000
75%,124.790000,128.808750,130.117083,125.590000,130.423333,135.996250,1.000000,1.000000,3.000000,1.000000,2.000000,2.0000
max,476.882449,144.890000,145.890000,145.423333,146.890000,150.123333,3.000000,11.090452,5.000000,38.747484,3.000000,2.0000


In [304]:
y = data[['lat', 'lon']]
y.describe()

,lat,lon
count,1500.000000,1500.000000
mean,-8.070097,-34.894814
std,0.004232,0.004354
min,-8.077570,-34.906830
25%,-8.073197,-34.897387
50%,-8.069940,-34.895346
75%,-8.067113,-34.891169
max,-8.059339,-34.885067


In [305]:
from sklearn.model_selection import train_test_split

In [306]:
from sklearn.preprocessing import RobustScaler

transformerY = RobustScaler().fit(y)
y = transformerY.transform(y)

print(y)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

[[-0.90964375 -0.15182341]
 [-0.49562395 -0.20521893]
 [-0.45223323 -0.17321378]
 ...
 [-1.20006574 -0.4984118 ]
 [-1.07433127 -0.70218327]
 [-0.49233677 -0.04631901]]


In [307]:
from sklearn.multioutput import MultiOutputRegressor

In [308]:
multioutputregressor = MultiOutputRegressor(model_xgb).fit(X_train, y_train)

In [309]:
print (transformerY.inverse_transform(multioutputregressor.predict(X_test)))

[[ -8.064641  -34.889904 ]
 [ -8.069117  -34.89561  ]
 [ -8.075045  -34.891617 ]
 [ -8.069219  -34.890816 ]
 [ -8.075997  -34.894936 ]
 [ -8.073624  -34.896885 ]
 [ -8.073148  -34.89627  ]
 [ -8.071207  -34.892464 ]
 [ -8.072372  -34.892544 ]
 [ -8.072586  -34.900692 ]
 [ -8.06942   -34.89713  ]
 [ -8.0694895 -34.893803 ]
 [ -8.077645  -34.899048 ]
 [ -8.070943  -34.89204  ]
 [ -8.072823  -34.890903 ]
 [ -8.067698  -34.895527 ]
 [ -8.070223  -34.893154 ]
 [ -8.072928  -34.894714 ]
 [ -8.06552   -34.88619  ]
 [ -8.071093  -34.891724 ]
 [ -8.064039  -34.885986 ]
 [ -8.072282  -34.896366 ]
 [ -8.069791  -34.890705 ]
 [ -8.070014  -34.891422 ]
 [ -8.06288   -34.903492 ]
 [ -8.072583  -34.897076 ]
 [ -8.0697    -34.88955  ]
 [ -8.069574  -34.890553 ]
 [ -8.074606  -34.899864 ]
 [ -8.067149  -34.896214 ]
 [ -8.072674  -34.896046 ]
 [ -8.06967   -34.88955  ]
 [ -8.065853  -34.889786 ]
 [ -8.067566  -34.89383  ]
 [ -8.070193  -34.899094 ]
 [ -8.068749  -34.891747 ]
 [ -8.065532  -34.885807 ]
 

In [310]:
print (np.mean((transformerY.inverse_transform(multioutputregressor.predict(X_test)) - transformerY.inverse_transform(y_test))**2, axis=0))

[3.31449974e-07 4.62825814e-07]


In [311]:
from math import sin, cos, sqrt, atan2, radians


def calculateDistance(lat1, lon1, lat2, lon2):
    R = 6373.0
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

In [317]:
v = transformerY.inverse_transform(multioutputregressor.predict(X_test))
print(v)

[[ -8.064641  -34.889904 ]
 [ -8.069117  -34.89561  ]
 [ -8.075045  -34.891617 ]
 [ -8.069219  -34.890816 ]
 [ -8.075997  -34.894936 ]
 [ -8.073624  -34.896885 ]
 [ -8.073148  -34.89627  ]
 [ -8.071207  -34.892464 ]
 [ -8.072372  -34.892544 ]
 [ -8.072586  -34.900692 ]
 [ -8.06942   -34.89713  ]
 [ -8.0694895 -34.893803 ]
 [ -8.077645  -34.899048 ]
 [ -8.070943  -34.89204  ]
 [ -8.072823  -34.890903 ]
 [ -8.067698  -34.895527 ]
 [ -8.070223  -34.893154 ]
 [ -8.072928  -34.894714 ]
 [ -8.06552   -34.88619  ]
 [ -8.071093  -34.891724 ]
 [ -8.064039  -34.885986 ]
 [ -8.072282  -34.896366 ]
 [ -8.069791  -34.890705 ]
 [ -8.070014  -34.891422 ]
 [ -8.06288   -34.903492 ]
 [ -8.072583  -34.897076 ]
 [ -8.0697    -34.88955  ]
 [ -8.069574  -34.890553 ]
 [ -8.074606  -34.899864 ]
 [ -8.067149  -34.896214 ]
 [ -8.072674  -34.896046 ]
 [ -8.06967   -34.88955  ]
 [ -8.065853  -34.889786 ]
 [ -8.067566  -34.89383  ]
 [ -8.070193  -34.899094 ]
 [ -8.068749  -34.891747 ]
 [ -8.065532  -34.885807 ]
 

In [318]:
lat1 = v[:,0]
lon1 = v[:,1]
lat2 = transformerY.inverse_transform(y_test)[:,0]
lon2 = transformerY.inverse_transform(y_test)[:,1]

In [319]:
dist = calculateDistance(lat1, lon2, lat2, lon2)

In [320]:
print(dist.mean())

2.7194509832946108


In [321]:
from sklearn import svm
svr = svm.SVR()
regr = MultiOutputRegressor(svr)
regr.fit(X_train,y_train)

MultiOutputRegressor(estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
           n_jobs=1)

In [292]:
out2 = regr.predict(X_test)
print(out2)

[[ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681524]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681657]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.684455   -348.95681537]
 [ -80.6

In [322]:
lat3 = out2[:,0]/10.0
lon3 = out2[:,1]/10.0
lat4 = y_test["lat"].values/10.0
lon4 = y_test['lon'].values/10.0

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [294]:
dist2 = calculateDistance(lat3, lon3, lat4, lon4)
print(dist.mean())

10.348545317122081
